# **IMPORT LIBRARY**

In [25]:
# Import Library
import pandas as pd
## Preprocessing text
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# **LOAD DATASET**

In [26]:
# Load file CSV hasil scraping
url = "https://raw.githubusercontent.com/heriswaya/Sentiment-Analysis-Dicoding/main/ulasan_alibaba.csv"

# Load data ke DataFrame
app_reviews_df = pd.read_csv(url)

# Tampilkan informasi awal
print("Jumlah ulasan:", app_reviews_df.shape[0])
app_reviews_df.head()

Jumlah ulasan: 99000


,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,27bf1c05-2d55-434b-8008-790a160016cd,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,The app is great. I run a pretty good sized co...,5,5262,8.82.0,2025-03-15 18:38:47,NaN,NaN,8.82.0
1,69f9cd05-17e7-4459-92dd-b915fe959e0f,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,The app doesn't work properly. Firstly it tell...,2,1985,8.77.0,2025-01-25 21:37:16,We are sorry for your experience. If you have ...,2025-02-06 02:20:34,8.77.0
2,1d6cb16d-c070-4db3-8dc9-8bb7ba86cb37,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,The apps functionality seriously sucks.... Sea...,1,3539,8.71.1,2024-12-02 13:06:25,We are sorry for your experience. If you have ...,2024-12-03 01:23:10,8.71.1
3,c83c1e64-6aa3-42e8-9a56-0385a297b87b,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Buyer beware! They have tons of listings that ...,1,2871,7.26.2,2021-01-25 04:11:00,NaN,NaN,7.26.2
4,ef5a3285-4849-4fe8-8855-e8315f07ebd9,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,"The items are great, but when I click on one o...",2,15608,8.28.0,2023-10-23 05:10:01,"Hi, we are sorry to hear that. Do share additi...",2023-11-10 11:05:44,8.28.0


In [27]:
app_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99000 entries, 0 to 98999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              99000 non-null  object
 1   userName              99000 non-null  object
 2   userImage             99000 non-null  object
 3   content               99000 non-null  object
 4   score                 99000 non-null  int64 
 5   thumbsUpCount         99000 non-null  int64 
 6   reviewCreatedVersion  75808 non-null  object
 7   at                    99000 non-null  object
 8   replyContent          10441 non-null  object
 9   repliedAt             10441 non-null  object
 10  appVersion            75808 non-null  object
dtypes: int64(2), object(9)
memory usage: 8.3+ MB


In [28]:
# Membuat DataFrame baru (clean_df) dengan menghapus baris yang memiliki nilai yang hilang (NaN) dari app_reviews_df
clean_df = app_reviews_df.dropna()

# Menghapus baris duplikat dari DataFrame clean_df
clean_df = clean_df.drop_duplicates()

# Menghitung jumlah baris dan kolom dalam DataFrame clean_df setelah menghapus duplikat
jumlah_ulasan_setelah_hapus_duplikat, jumlah_kolom_setelah_hapus_duplikat = clean_df.shape

In [29]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6961 entries, 1 to 98948
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              6961 non-null   object
 1   userName              6961 non-null   object
 2   userImage             6961 non-null   object
 3   content               6961 non-null   object
 4   score                 6961 non-null   int64 
 5   thumbsUpCount         6961 non-null   int64 
 6   reviewCreatedVersion  6961 non-null   object
 7   at                    6961 non-null   object
 8   replyContent          6961 non-null   object
 9   repliedAt             6961 non-null   object
 10  appVersion            6961 non-null   object
dtypes: int64(2), object(9)
memory usage: 652.6+ KB


# **PREPROCESSING TEXT**

In [30]:
# Download data NLTK
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [31]:
# Inisialisasi stemmer dan stopwords
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

In [32]:
# Function: Membersihkan teks
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)  # remove mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text)  # remove hashtags
    text = re.sub(r'RT[\s]+', '', text)        # remove RT
    text = re.sub(r"http\S+", '', text)        # remove links
    text = re.sub(r'[0-9]+', '', text)         # remove numbers
    text = re.sub(r'[^\x00-\x7F]+', ' ', text) # remove non-ASCII (emoji, simbol asing)
    text = text.translate(str.maketrans('', '', string.punctuation)) # remove punctuation
    return text

# Funtion: Mengubah semua karakter dalam teks menjadi huruf kecil
def casefoldingText(text):
    text = text.lower()
    return text

# Function: Tokenizing
def tokenizingText(text):
    return word_tokenize(text)

# Function: Stopword removal
def filteringText(tokens):
    return [word for word in tokens if word not in stop_words]

# Function: Stemming
def stemmingText(tokens):
    return [stemmer.stem(word) for word in tokens]

# Function: Ubah token jadi kalimat lagi
def toSentence(tokens):
    return ' '.join(tokens)

# Function: Ubah slangwords ke kata aslinya
slangwords = {
    # Common slang
    "u": "you",
    "ur": "your",
    "r": "are",
    "ya": "you",
    "thx": "thanks",
    "pls": "please",
    "im": "i am",
    "idk": "i do not know",
    "lol": "laugh out loud",
    "btw": "by the way",
    "omg": "oh my god",
    "b4": "before",
    "cuz": "because",
    "wanna": "want to",
    "gonna": "going to",
    "gotta": "got to",
    "aint": "am not",
    "luv": "love",

    # Extended list
    "2day": "today",
    "2moro": "tomorrow",
    "2nite": "tonight",
    "4ever": "forever",
    "4u": "for you",
    "afaik": "as far as i know",
    "asap": "as soon as possible",
    "atm": "at the moment",
    "b": "be",
    "bc": "because",
    "bday": "birthday",
    "brb": "be right back",
    "bro": "brother",
    "bros": "brothers",
    "cm": "call me",
    "dunno": "do not know",
    "dwiw": "deal with it",
    "fyi": "for your information",
    "gg": "good game",
    "gl": "good luck",
    "gr8": "great",
    "hbu": "how about you",
    "hf": "have fun",
    "hmu": "hit me up",
    "ic": "i see",
    "icymi": "in case you missed it",
    "idc": "i do not care",
    "ikr": "i know right",
    "ily": "i love you",
    "imo": "in my opinion",
    "irl": "in real life",
    "jk": "just kidding",
    "k": "okay",
    "kk": "okay",
    "lemme": "let me",
    "m8": "mate",
    "msg": "message",
    "n": "and",
    "nvm": "never mind",
    "ofc": "of course",
    "omw": "on my way",
    "pov": "point of view",
    "ppl": "people",
    "qt": "cutie",
    "rofl": "rolling on the floor laughing",
    "sry": "sorry",
    "sup": "what's up",
    "tbh": "to be honest",
    "tmi": "too much information",
    "ttyl": "talk to you later",
    "txt": "text",
    "ty": "thank you",
    "tyt": "take your time",
    "w/": "with",
    "w/o": "without",
    "wbu": "what about you",
    "wyd": "what you doing",
    "xoxo": "hugs and kisses",
    "y": "why",
    "yolo": "you only live once",
    "yw": "you're welcome",

    # Alibaba/e-commerce specific
    "app": "application",
    "plz": "please",
    "thnk": "think",
    "ordr": "order",
    "prod": "product",
    "svc": "service",
    "cust": "customer",
    "deliv": "delivery",
    "disc": "discount",
    "refund": "refund",
    "ship": "shipping",
    "paymnt": "payment",
    "acct": "account",
    "promo": "promotion",
    "qty": "quantity",
    "rply": "reply",
    "trans": "transaction",
    "vry": "very",
    "w8": "wait"
}
def fix_slangwords(text):
    words = text.split()
    fixed_words = []
    for word in words:
        fixed_words.append(slangwords.get(word.lower(), word))
    return ' '.join(fixed_words)

In [33]:
# Membersihkan teks dan menyimpannya di kolom 'text_clean'
clean_df['text_clean'] = clean_df['content'].apply(cleaningText)

# Mengubah huruf dalam teks menjadi huruf kecil dan menyimpannya di 'text_casefoldingText'
clean_df['text_casefoldingText'] = clean_df['text_clean'].apply(casefoldingText)

# Mengganti kata-kata slang dengan kata-kata standar dan menyimpannya di 'text_slangwords'
clean_df['text_slangwords'] = clean_df['text_casefoldingText'].apply(fix_slangwords)

# Memecah teks menjadi token (kata-kata) dan menyimpannya di 'text_tokenizingText'
clean_df['text_tokenizingText'] = clean_df['text_slangwords'].apply(tokenizingText)

# Menghapus kata-kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
clean_df['text_stopword'] = clean_df['text_tokenizingText'].apply(filteringText)

# Menggabungkan token-token menjadi kalimat dan menyimpannya di 'text_akhir'
clean_df['text_akhir'] = clean_df['text_stopword'].apply(toSentence)

In [34]:
clean_df

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir
1,69f9cd05-17e7-4459-92dd-b915fe959e0f,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,The app doesn't work properly. Firstly it tell...,2,1985,8.77.0,2025-01-25 21:37:16,We are sorry for your experience. If you have ...,2025-02-06 02:20:34,8.77.0,The app doesnt work properly Firstly it tells ...,the app doesnt work properly firstly it tells ...,the application doesnt work properly firstly i...,"[the, application, doesnt, work, properly, fir...","[application, doesnt, work, properly, firstly,...",application doesnt work properly firstly tells...
2,1d6cb16d-c070-4db3-8dc9-8bb7ba86cb37,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,The apps functionality seriously sucks.... Sea...,1,3539,8.71.1,2024-12-02 13:06:25,We are sorry for your experience. If you have ...,2024-12-03 01:23:10,8.71.1,The apps functionality seriously sucks Searchi...,the apps functionality seriously sucks searchi...,the apps functionality seriously sucks searchi...,"[the, apps, functionality, seriously, sucks, s...","[apps, functionality, seriously, sucks, search...",apps functionality seriously sucks searching i...
4,ef5a3285-4849-4fe8-8855-e8315f07ebd9,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,"The items are great, but when I click on one o...",2,15608,8.28.0,2023-10-23 05:10:01,"Hi, we are sorry to hear that. Do share additi...",2023-11-10 11:05:44,8.28.0,The items are great but when I click on one of...,the items are great but when i click on one of...,the items are great but when i click on one of...,"[the, items, are, great, but, when, i, click, ...","[items, great, click, one, variation, options,...",items great click one variation options add en...
6,7a65dce8-3f09-4e4e-a263-55efebc13c65,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,It's all around a great app except for the fac...,4,4342,6.10.2,2018-12-21 18:25:23,Thanks for your feedback. Could you tell us mo...,2018-01-10 06:01:58,6.10.2,Its all around a great app except for the face...,its all around a great app except for the face...,its all around a great application except for ...,"[its, all, around, a, great, application, exce...","[around, great, application, except, face, sen...",around great application except face send file...
14,b3984ad2-24e5-4a6a-9f50-08c93a4bc7b6,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,I concur that this app manipulates the audio d...,1,822,7.44.0,2021-11-19 03:42:39,The problem you reported has been fixed. Pleas...,2021-11-29 06:32:48,7.44.0,I concur that this app manipulates the audio d...,i concur that this app manipulates the audio d...,i concur that this application manipulates the...,"[i, concur, that, this, application, manipulat...","[concur, application, manipulates, audio, driv...",concur application manipulates audio driver st...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98943,bf42df3e-c0f6-4398-8d64-2227ac29a57a,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Nice,2,0,8.73.0,2024-12-18 15:37:00,"Thank you for using Alibaba.com App , any quer...",2024-12-18 16:00:02,8.73.0,Nice,nice,nice,[nice],[nice],nice
98944,70bc15ae-f336-47fb-91af-8b04e6290e01,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Cool,2,0,8.73.0,2025-01-01 10:41:16,"Thank you for using Alibaba.com App , any quer...",2025-01-01 12:00:12,8.73.0,Cool,cool,cool,[cool],[cool],cool
98945,b357ce96-6248-4aeb-9bd4-319031eacfe7,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Great,2,0,8.73.0,2024-12-17 18:06:18,"Thank you for using Alibaba.com App , any quer...",2024-12-17 19:00:07,8.73.0,Great,great,great,[great],[great],great
98946,fdfc3e21-1192-4465-9227-d957414d099f,A Google user,https://play-lh.googleusercontent.com/EGemo